# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/reversed_hgru_with_corr_1 - Canada/model/')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import random
from scipy import stats
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score

from pipeline_config import *
from reversed_hgru_data_preprocess.preprocess_config import *

#from Prices.prices_over_time import *
import os

In [3]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [4]:
hgru_test_preds_path = '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model_norway/test_predictions.pickle'

# Read Results Data

In [5]:
with open(test_predictions_path_with_hgru, 'rb') as f:
    prediction_dic = pickle.load(f)

with open(hgru_test_preds_path, 'rb') as f:
    hgru_prediction_dic = pickle.load(f)

with open(norway_path, 'rb') as f:
    raw_dataset_dict = pickle.load(f)
    
with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [6]:
def create_test_dataframe(raw_dataset_dict: dict):
    test_dict = {}
    for key in raw_dataset_dict.keys():
        df = raw_dataset_dict[key][['Category', 'Date', 'Year', 'Indent', 'Inflation t+1']]
        df.dropna(inplace=True)
        df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
        target_df = df[df['Year'] > Year]
        test_dict[key] = target_df
    return test_dict

test_dict = create_test_dataframe(raw_dataset_dict)


/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_5797/1314998454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_5797/1314998454.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Inflation t+1': 'Actual'}, inplace=True)
/var/folders/88/mkn0vj0s1kl1179x22s6j2h80000gq/T/ipykernel_5797/1314998454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [7]:
def get_df_with_predictions(prediction_dic: dict, dict_of_categories_df:dict) -> dict:
    all_data_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        prediction_df.rename(columns = {0: 'Prediction'}, inplace=True)
        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)
        all_data_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df], axis=1)
    return all_data_dict

all_data_test_dict=get_df_with_predictions(prediction_dic, test_dict)

In [8]:
len(all_data_test_dict.keys())

52

In [9]:
categories_per_indent_dict ={key: categories_per_indent_dict[key] for key in categories_per_indent_dict.keys() if key in [0,1,2,3]}
cat_ids = list(categories_per_indent_dict.values())

flat_cat_id = [item for sublist in cat_ids for item in sublist]

all_categories = []
for cat_id in flat_cat_id:
    all_categories.append(category_id_to_name_dict[cat_id])

#all_categories = list(all_data_test_dict.keys())

In [10]:
all_categories = list(all_data_test_dict.keys())

In [11]:
len(all_categories)

52

In [12]:
def plot_results(all_data_dict, categories):
    category_samples = random.sample(categories, 20)+['All-items']
    #category_samples = categories
    for category in category_samples:
        category_df = all_data_dict[category]
        y_pred = category_df['Prediction'].values
        y_actual = category_df['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        print(f'Category is: {category}')
        print(f'RMSE is: {np.sqrt(mse)}')

        fig = px.line(category_df, x="Date", y=["Actual", "Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

#plot_results(all_data_test_dict, all_categories)

In [13]:
def avg_rmse(all_data_test_dict):
    mse_lst = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)
        mse_lst.append(mse)
    
    rmse_list = list(map(np.sqrt,mse_lst))
    avg_rmse = np.mean(rmse_list)
    rmse_std = np.std(rmse_list)
    avg_mse = np.mean(mse_lst)
    mse_std = np.std(mse_lst)
    
    print(f'RMSE:  {avg_rmse}')
    print(f'RMSE std:  {rmse_std}')
    print(f'MSE:  {avg_mse}')
    print(f'MSE std:  {mse_std}')
    #print(f'interval: {[avg_rmse-rmse_std, avg_rmse+rmse_std]}')

    return avg_rmse,rmse_std

In [14]:
avg_rmse(all_data_test_dict)

RMSE:  1.3260963118531572
RMSE std:  1.2708645041268356
MSE:  3.373628016160093
MSE std:  8.461075141676984


(1.3260963118531572, 1.2708645041268356)

In [15]:
def total_corr(all_data_test_dict):
    corr_dict = {}
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        corr = stats.pearsonr(y_pred,y_actual)[0]
        corr_dict[key] =  corr
    
    total_corr = sum(corr_dict.values())
    #avg_corr = total_corr/len(list(all_data_test_dict.keys()))
    
    num_high_corr = 0
    for category in corr_dict:
        if corr_dict[category] >= 0.5:
            num_high_corr +=1
    
    print(f'Number of categories with High Correlation: {num_high_corr}')
    
    return total_corr

In [16]:
total_corr(all_data_test_dict)

Number of categories with High Correlation: 31


28.42892813990584

In [17]:
def avg_r_squared(all_data_test_dict):
    r_squared_lst = []
    low_r_squared_indent = []
    positive_r_squared_indent = []
    high_r_squared_indent = []
    for key in all_categories:
        df_predictions = all_data_test_dict[key]
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        r2 = r2_score(y_actual, y_pred)
        r_squared_lst.append(r2)

        if r2<=0:
            low_r_squared_indent.append(df_predictions['Indent'].unique().item())
        
        else:
            positive_r_squared_indent.append(df_predictions['Indent'].unique().item())


        if r2>=0.5:
            high_r_squared_indent.append(df_predictions['Indent'].unique().item())

        if key =='All-items':
            headline_r2 = r2
    
    avg_r2_score = np.mean(r_squared_lst)
    r2_std = np.std(r_squared_lst)
    
    print(f'Average R Squared:  {avg_r2_score}')
    print(f'R Squared std:  {r2_std}')
    print(f'interval: {[avg_r2_score-r2_std, avg_r2_score+r2_std]}')
    print('--------------------------------------------------------')
    #print(f'sorted r2 list: {sorted(r_squared_lst, reverse=True)}')
    print(f'Headline R2: {headline_r2}')
    print(f'R2 list percentiles:\n[10: {np.percentile(r_squared_lst, 10)}, 25: {np.percentile(r_squared_lst, 25)}, 50: {np.percentile(r_squared_lst, 50)}, 75: {np.percentile(r_squared_lst, 75)}, 90: {np.percentile(r_squared_lst, 90)}]')
    print(f'num of neg/zero R^2: {sum(1 for i in r_squared_lst if i<= 0)}')
    print(f'num of postive R^2: {sum(1 for i in r_squared_lst if i> 0)}')
    print(f'num of high R^2: {sum(1 for i in r_squared_lst if i>= 0.5)}')


    #return headline_r2, low_r_squared_indent, positive_r_squared_indent, high_r_squared_indent

In [18]:
avg_r_squared(all_data_test_dict)

Average R Squared:  0.2861136389726585
R Squared std:  0.2836962399346178
interval: [0.002417399038040724, 0.5698098789072763]
--------------------------------------------------------
Headline R2: 0.2643776004212751
R2 list percentiles:
[10: -0.028179705792179315, 25: 0.0860044723379774, 50: 0.22704006820645078, 75: 0.5255859186527247, 90: 0.7159900301330198]
num of neg/zero R^2: 6
num of postive R^2: 46
num of high R^2: 14


In [19]:
len(all_categories)

52

In [20]:
plot_results(all_data_test_dict, all_categories)

Category is: Insurance
RMSE is: 0.30139097797309433


Category is: Transport services
RMSE is: 3.3988873846380114


Category is: Telephone services
RMSE is: 0.5649979629457074


Category is: Footwear
RMSE is: 1.190603475851709


Category is: Other services n.e.c.
RMSE is: 1.3216104618584519


...

Category is: Household appliances
RMSE is: 2.08291675077075


Category is: Alcoholic beverages and tobacco
RMSE is: 0.5164783488673548


Category is: Restaurants and hotels
RMSE is: 0.5757448219759276


Category is: Package holidays
RMSE is: 1.3941802914742631


Category is: All-items
RMSE is: 0.47225204076307736


In [21]:
def create_rmse_table(all_data_test_dict):
    categories = []
    indents = []
    mses = []
    rmses = []
    for key in all_categories:
        categories.append(key)

        df_predictions = all_data_test_dict[key]
        indents.append(df_predictions.Indent.unique().item())
        y_pred = df_predictions['Prediction'].values
        y_actual = df_predictions['Actual'].values
        mse = mean_squared_error(y_pred, y_actual)

        mses.append(mse)
        rmses.append(np.sqrt(mse))
    
    series1 = pd.DataFrame(categories).reset_index(drop=True).rename(columns={0:'Category'})
    series2 = pd.DataFrame(indents).reset_index(drop=True).rename(columns={0:'Indent'})
    series3 = pd.DataFrame(mses).reset_index(drop=True).rename(columns={0:'MSE'})
    series4 = pd.DataFrame(rmses).reset_index(drop=True).rename(columns={0:'RMSE'})

    df1 = pd.concat([series1, series2], axis = 1)
    df2 = pd.concat([df1, series3], axis = 1)
    df3 = pd.concat([df2, series4], axis = 1)

    return df3


In [22]:
from pathlib import Path
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

In [23]:
path = os.getcwd()

desired_path = path + '/rmse_per_category_norway.csv'

rmse_per_cat = create_rmse_table(all_data_test_dict)
rmse_per_cat.to_csv(desired_path, index=False)

In [24]:
rmse_per_cat.sort_values('Indent')

,Category,Indent,MSE,RMSE
51,All-items,0,0.223022,0.472252
39,Alcoholic beverages and tobacco,1,0.266750,0.516478
40,Clothing and footwear,1,2.445978,1.563962
41,Communications,1,0.645015,0.803128
42,Education,1,0.017048,0.130566
43,Food and non-alcoholic beverages,1,1.370190,1.170551
50,Transport,1,1.463490,1.209748
45,Health,1,0.136896,0.369995
46,"Housing, water, electricity, gas and other fuels",1,1.809053,1.345011
47,Miscellaneous goods and services,1,0.210894,0.459232


In [25]:
def get_df_with_predictions_with_hgru(prediction_dic: dict, hgru_pred_dic: dict, dict_of_categories_df:dict) -> dict:
    bidirect_hgru_dict = {}
    for key in list(prediction_dic.keys()):
        predictions = prediction_dic[key]
        prediction_df =  pd.DataFrame(predictions.flatten().detach().numpy())
        hgru_predictions = hgru_pred_dic[key]
        hgru_prediction_df =  pd.DataFrame(hgru_predictions.flatten().detach().numpy())

        prediction_df.rename(columns = {0: 'Bidirectional_Prediction'}, inplace=True)
        hgru_prediction_df.rename(columns = {0: 'HGRU_Prediction'}, inplace=True)

        dict_of_categories_df[key] = dict_of_categories_df[key].reset_index(drop=True)

        bidirect_hgru_dict[key] = pd.concat([dict_of_categories_df[key], prediction_df, hgru_prediction_df], axis=1)

    return bidirect_hgru_dict

bidirect_hgru_dict=get_df_with_predictions_with_hgru(prediction_dic, hgru_prediction_dic, test_dict)

In [26]:
def plot_results_bidirect_hgru(bidirect_hgru_dict, categories):
    #category_samples = random.sample(categories, 20)+['All items']
    category_samples = ['All-items'] + categories
    for category in category_samples:
        category_df = bidirect_hgru_dict[category]
        y_pred_bidirect = category_df['Bidirectional_Prediction'].values
        y_pred_hgru = category_df['HGRU_Prediction'].values
        y_actual = category_df['Actual'].values
        bidirect_mse = mean_squared_error(y_pred_bidirect, y_actual)
        hgru_mse = mean_squared_error(y_pred_hgru, y_actual)
        print(f'Category is: {category}')
        print(f'Bidirectional RMSE is: {np.sqrt(bidirect_mse)}')
        print(f'HGRU RMSE is: {np.sqrt(hgru_mse)}')

        fig = px.line(category_df, x="Date", y=["Actual", "Bidirectional_Prediction", "HGRU_Prediction"], title=f'{category} - Actual VS Prediction')
        fig.show()

#plot_results(all_data_test_dict, all_categories)

In [27]:
categories_with_highest_diff = [
    'Transport services',
'Non-alcoholic beverages',
'Household textiles',
'Glassware, tableware and household utensils',
'Furniture and furnishings, carpets and other floor coverings',
'Maintenance and repair of the dwelling',
'Newspapers, books and stationery',
'Clothing and footwear',
'Package holidays',
'Goods and services for routine household maintenance',
'Food and non-alcoholic beverages',
'Electricity, gas and other fuels',
'Footwear',
'Other major durables for recreation and culture',
'Accommodation services',
'Furnishings, household equipment and routine maintenance',
'Actual rentals for housing',
'Personal effects n.e.c.',
'Food',
'Imputed rentals for housing',
'Water supply and miscellaneous services relating to the dwelling',
'Household appliances',
'Clothing',
'Alcoholic beverages and tobacco',
'Telephone equipment',
'Medical products, appliances and equipment',
'Transport',
'Tobacco',
'Recreational and cultural services',
'Restaurant services',
'Other services n.e.c.',
'Alcoholic beverages',
'Operation of personal transport equipment',
'Education'
]

In [28]:
plot_results_bidirect_hgru(bidirect_hgru_dict, categories_with_highest_diff)

Category is: All-items
Bidirectional RMSE is: 0.47225204076307736
HGRU RMSE is: 0.5431330937733596


Category is: Transport services
Bidirectional RMSE is: 3.3988873846380114
HGRU RMSE is: 7.07149029192674


Category is: Non-alcoholic beverages
Bidirectional RMSE is: 1.9282832983025964
HGRU RMSE is: 3.8116931824639866


Category is: Household textiles
Bidirectional RMSE is: 4.753235711632928
HGRU RMSE is: 6.19930711292136


Category is: Glassware, tableware and household utensils
Bidirectional RMSE is: 2.1995387334047387
HGRU RMSE is: 3.310812056329825


...

Category is: Restaurant services
Bidirectional RMSE is: 0.3737306248779452
HGRU RMSE is: 0.6429705353934995


Category is: Other services n.e.c.
Bidirectional RMSE is: 1.3216104618584519
HGRU RMSE is: 1.5739657738526451


Category is: Alcoholic beverages
Bidirectional RMSE is: 0.4686530528240121
HGRU RMSE is: 0.7195906687799305


Category is: Operation of personal transport equipment
Bidirectional RMSE is: 2.15228407472945
HGRU RMSE is: 2.3520449480367582


Category is: Education
Bidirectional RMSE is: 0.13056638198248116
HGRU RMSE is: 0.31554295119229503
